# Введение в машинное обучение

## Семинар #8

### Екатерина Кондратьева

ekaterina.kondrateva@skoltech.ru

## Отбор и генерация признаков (Feature Engineering). Поиск и оптимизация модели (Grid Search). Поиск аномалий (Anomaly Detection)

In [ ]:
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline

## Кто такие аутлайеры (аномалии)? 

Это наблюдение, которое "**отличается**" "**настолько сильно**" от "**прочих наблюдений**"

Определение аномалий в датасете невозможно без: 
   1. определения **"прочих наблюдений"** (модели данных)
   2. определения метрики отклонения (**"отличия"**) от **"прочих наблюдений"** 
   3. определения **"насколько сильно"** аномальные объекты **"отличаются"** от **"прочих наблюдений"**

##### Назовите самый простой способ детектирования аномалий?

*Спойлер: СТД? Квантили?*

## 1.  Нахождение фрода

Источник: https://www.kaggle.com/dalpozz/creditcardfraud

In [ ]:
data = pd.read_csv("./data/creditcard.csv")

Давайте проверим, сколько "фрода" в этой выборке?

In [ ]:
data.head()

In [ ]:
# <YOUR CODE>

Должно быть  1.7% , что можно сказать про баланс классов?

In [ ]:
data.describe()

In [ ]:
full_X = data.drop(columns="Class")
full_y = data["Class"]

Будем делать **красиво**:
    
    - отложим заранее из оригинальной выборки тестовую, размером 0.4
    - будем выбирать модель кросс валидацией на выборке `train`
    - не забудем про стратификацию

In [ ]:
from sklearn.model_selection import train_test_split

full_X.reset_index(drop=True)

tt_split = train_test_split(full_X, full_y, test_size=0.4, stratify=full_y)#<YOUR CODE>

train_X, test_X, train_y, test_y = tt_split 

Будем делать **красиво**, выберем несколько метрик:
    
    1. стандартно `accuracy`, почему это плохо в нашем случае?
    2. `precision_score` или доля истинных 1 среди всех помеченных как 1
    3. `recall_score` или полнота "вероятность", с которой все истинные 1 предсказаны как 1
    4. `fbeta_score` - частный случай которого, наш знакомый `f1_score` 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, fbeta_score, f1_score

In [ ]:
f1_score?

In [ ]:
fbeta_score?

Зачем нам $\beta$ ? : https://en.wikipedia.org/wiki/F1_score

Возьмем $\beta = 1.7$ как будто, мы заранее знаем, сколько нам ожидать ффрода в выборке.

In [ ]:
def show_metrics(y_true, y_pred):
    return pd.Series(
        {
        "accuracy": accuracy_score(y_true, y_pred),
        
        "precision": precision_score(y_true, y_pred),
        
        "recall": recall_score(y_true, y_pred),
        
        "f1_score": f1_score(y_true, y_pred),
        
        "f_0.2": fbeta_score(y_true, y_pred, beta=.17),
    }
)

## Обнаружение выбросов

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve

def plot_level_lines(model, data, size=100):
    def _expand(a, b, frac=.5, margin=1.):
        return a - abs(a) * frac - margin, b + abs(b) * frac + margin

    # Вспомогательная функция для рисования линий уровня и набора точек
    plt.figure(figsize=(10, 7))
    min_x, min_y = data.min(axis=0)
    max_x, max_y = data.max(axis=0)
    min_x, max_x = _expand(min_x, max_x)
    min_y, max_y = _expand(min_y, max_y)

    # создаём регулярную сетку для контуров
    all_x = np.linspace(min_x, max_x, num=size)
    all_y = np.linspace(min_y, max_y, num=size)
    XX, YY = np.meshgrid(all_x, all_y)
    test_data = np.c_[XX.ravel(), YY.ravel()]

    # опрашиваем предсказания модели
    try:
        predictions = model.decision_function(test_data).reshape(size, size)
        data_scores = model.predict(data)
        anomaly_scores = model.decision_function(data)

    except AttributeError:
        predictions = model._decision_function(test_data).reshape(size, size)
        data_scores = model._predict(data)
        anomaly_scores = model._decision_function(data)

    # создаём график контуров с заливкоц
    plt.contourf(all_x, all_y, predictions, cmap=plt.cm.coolwarm)

    # отображаем границу принятия решений
    threshold = anomaly_scores[data_scores==1.0].min()
    plt.contour(XX, YY, predictions, levels=[threshold], linewidths=2, colors='darkred')

    # нарисуем точки выборки
    plt.scatter(data[:, 0], data[:, 1])

    axes = plt.gca()
    axes.set_xlim([min_x,max_x])
    axes.set_ylim([min_y,max_y])

    plt.show()
    plt.close()

Эта процедура рисует кривые ROC и Precision-Recalls.

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve


def plot_results(y_test, probabilities):
    # построим кривые ROC и PR
    fpr, tpr, _ = roc_curve(y_test, probabilities)
    
    # roc
    plt.plot(fpr, tpr, linewidth=3)
    plt.xlabel('FPR')
    plt.ylabel('TPR')

    plt.grid(True)
    plt.show()

Создаём несколько скоплений точек и подмешиваем аномальных точек

In [ ]:
from sklearn.datasets import make_blobs


def data_generator(n_samples=100, anomaly_fraction=0.1, n_features=2):
    n_anomaly = int(n_samples * anomaly_fraction)
    n_normal = n_samples - n_anomaly

    normal_data, _ = make_blobs(n_normal, n_features=n_features, centers=3)

    anomaly_data = np.random.rand(n_anomaly, n_features)

    nrm_min = normal_data.min(axis=0).reshape(1, -1)
    nrm_ptp = normal_data.ptp(axis=0).reshape(1, -1)
    anomaly_data = anomaly_data * nrm_ptp + nrm_min

    return np.concatenate([normal_data, anomaly_data], axis=0)

Создаём одно скопление точек, к которому подмешаны аномалии

In [ ]:
def unimodal_data_generator(n_samples=100, anomaly_fraction=0.1, n_features=2):
    n_anomaly = int(n_samples * anomaly_fraction)
    n_normal = n_samples - n_anomaly

    xform = np.array([[2, 0.5], [0.5, 1.5]])
    normal_data = np.random.randn(n_normal, n_features) @ xform
    anomaly_data = np.random.rand(n_anomaly, n_features)

    nrm_min = normal_data.min(axis=0).reshape(1, -1)
    nrm_ptp = normal_data.ptp(axis=0).reshape(1, -1)
    anomaly_data = anomaly_data * nrm_ptp + nrm_min

    return np.concatenate([normal_data, anomaly_data], axis=0)

Сгенерируем данные

In [ ]:
data_blob = unimodal_data_generator()

data_blobs = data_generator()

<br>

### Эллиптическая Огибающая

Метод **Elliptic Envelope** оценивая ковариацию данныхпредполагает, что 
* данные порожденные эллиптическим распределением

* аномальные точки находятся дальше от центра скопления, чем нормальные

In [ ]:
from sklearn.covariance import EllipticEnvelope

In [ ]:
def cluster_plot(y_test, probas):
    # Fit the model
    plt.figure(figsize=(9, 7))


    clf.fit(X)
    scores_pred = clf.decision_function(X)
    y_pred = clf.predict(X)
    threshold = stats.scoreatpercentile(scores_pred, 100 * outliers_fraction)
    n_errors = (y_pred != ground_truth).sum()

    # plot the levels lines and the points
    Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)

    plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), threshold, 7),
                     cmap=plt.cm.Blues_r)
    a = plt.contour(xx, yy, Z, levels=[threshold],
                        linewidths=2, colors='red')
    plt.contourf(xx, yy, Z, levels=[threshold, Z.max()],
                     colors='orange')
    b = plt.scatter(X[:-n_outliers, 0], X[:-n_outliers, 1], c='white',
                        s=20, edgecolor='k')
    c = plt.scatter(X[-n_outliers:, 0], X[-n_outliers:, 1], c='black',
                        s=20, edgecolor='k')
    plt.axis('tight')
    plt.legend(
        [a.collections[0], b, c],
        ['learned decision function', 'true inliers', 'true outliers'],
        prop=matplotlib.font_manager.FontProperties(size=10),
        loc='lower right')
    plt.xlabel("%d. %s (errors: %d)" % (i + 1, 'Elliptic Envelope', n_errors))
    plt.xlim((-7, 7))
    plt.ylim((-7, 7))
    plt.suptitle("Outlier detection via Elliptic Envelope")

plt.show()

In [ ]:
model = EllipticEnvelope(assume_centered=False, contamination=0.5)

model.fit(data_blob)

plot_level_lines(model, data_blob)

In [ ]:
model = EllipticEnvelope(assume_centered=False, contamination=0.5)

model.fit(data_blobs)

plot_level_lines(model, data_blobs)

Плюсы:
* простой в использовании метод
* порождает интерпретируемую границу

Минусы:
* Годится только для **одномодальных** распределений
* Подвержен вырождению ковариационной матрицы

Посмотрим на его работу в обнаружении мошенничества

In [ ]:
model = EllipticEnvelope(contamination=0.01)

model.fit(train_X)

predictions_elliptic = -model.decision_function(test_X)

labels_elliptic = model.predict(test_X)

In [ ]:
plot_results(test_y, predictions_elliptic)

metrics_elliptic = show_metrics(test_y, (labels_elliptic < 0) * 1)

<br>

### Изолирующий Лес (Isolation Forest)

* для каждого наблюдения рассчитывается средняя длина пути до него в случайном дереве
  * деревья со случайными разбиениями с целью изолировать наблюдения
  
* чем короче средняя длина пути тем проще описать наблюдение
  * тем в более изолированной части пространства оно находится

* чем больше разбиений нужно для наблюдения, нормальнее оно

In [ ]:
from sklearn.ensemble import IsolationForest

In [ ]:
model = IsolationForest(n_estimators=100,
                        contamination=0.1,
                        max_features=1.0,
                        max_samples=1.0,
                        bootstrap=True,
                        random_state=0)
model.fit(data_blob)

plot_level_lines(model, data_blob)

Несколько скоплений

In [ ]:
model = IsolationForest(n_estimators=100,
                        contamination=0.1,
                        max_features=1.0,
                        max_samples=1.0,
                        bootstrap=True,
                        random_state=0)
model.fit(data_blobs)

plot_level_lines(model, data_blobs)

Плюсы:

* Робастный метод

Минусы:
* плохая интерпертация

* не различает скопления аномалий

In [ ]:
model = IsolationForest()

model.fit(train_X)

predictions_isolation = -model.decision_function(test_X)

labels_isolation = model.predict(test_X)

In [ ]:
plot_results(test_y, predictions_isolation)

metrics_isolation = show_metrics(test_y, (labels_isolation < 0) * 1)

<br>

## Local Outlier Factor

Основан на наблюдении, что нормальные наблюдения имеют тенденцию **скапливаться**

* вводится показатель локальной плотности, обратно пропорциональный средним расстоянием до $k$ ближайших соседей

* попарно сравнивается с показателями соседей

* вычисляется отношение локальной аномальности

Метод: https://towardsdatascience.com/local-outlier-factor-for-anomaly-detection-cc0c770d2ebe

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

![img](https://upload.wikimedia.org/wikipedia/commons/thumb/5/59/LOF.svg/400px-LOF.svg.png)

In [ ]:
model = LocalOutlierFactor(n_neighbors=20, 
                           contamination=0.1,
                           metric='minkowski', 
                           p=2)
model.fit(data_blob)

plot_level_lines(model, data_blob)

In [ ]:
model = LocalOutlierFactor(n_neighbors=20, 
                           contamination=0.1,
                           metric='minkowski', 
                           p=2)
model.fit(data_blobs)

plot_level_lines(model, data_blobs)

Плюсы:
* непараметрический метод

Минусы:
* подвержен проблеме "проклятия размерности", тк основан на расстояниях
* не может отличить скопления аномалий от нормальных точек
* без модификаций не поддерживает проверку аномальности на новых данных без обучения

In [ ]:
model = LocalOutlierFactor(contamination=0.02)

model.fit(train_X)

predictions_lof = -model._decision_function(test_X)

labels_lof = -predictions_lof

In [ ]:
plot_results(test_y, predictions_lof)

show_metrics(test_y, (labels_lof < 0) * 1)

## One Class SVM

Основная идея -- отделить данные в спрямляющем пространстве **мягкой гиперплоскостью** от нуля

Решает задачу
\begin{aligned}
  & \underset{\rho, f\in \mathcal{H}}{\text{минимизировать}}
    & & \tfrac12 \|f\|^2 - \rho
        + \tfrac1{m \nu} \sum_{i=1}^m \max\bigl\{
            0, \rho - f(x_i) \bigr\}\,,
\end{aligned}

Визуализация одноклассового метода опорных векторов: http://rvlasveld.github.io/blog/2013/07/12/introduction-to-one-class-support-vector-machines/

Визуализация `gamma` для `rbf` ядра: https://bitquill.net/blog/quick-hack-visualizing-rbf-bandwidth/

In [ ]:
from sklearn.svm import OneClassSVM

In [ ]:
model = OneClassSVM(nu=0.5, kernel='rbf', gamma=0.1)

model.fit(data_blobs)

plot_level_lines(model, data_blobs)

In [ ]:
model = OneClassSVM(nu=0.5, kernel='rbf', gamma=0.1)

model.fit(data_blob)

plot_level_lines(model, data_blob)

Плюсы:
* непараметрический метод
* применим не только к объектам из $\mathbb{R}^n$ (линейного пространства)
  * ядра на строках, графах и пр.
* может быть полезным при разумном выборе ядра

Минусы:
* вычислительно затратен и плохо масштабируется
* необходимо хранить часть обучающей выборки

In [ ]:
train_X.shape

In [ ]:
model = OneClassSVM(nu=0.017, kernel='rbf', gamma=0.1)

model.fit(train_X)

predictions_svm = -model.decision_function(test_X)

labels_svm = model.predict(test_X)

In [ ]:
plot_results(test_y, predictions_svm)

metrics_svm = show_metrics(test_y, (labels_svm < 0) * 1)

<br>

## Обнаружение аномалий методом PCA


In [ ]:
from sklearn.decomposition import PCA

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.decomposition import PCA


class PCAAnomalyDetector(BaseEstimator):
    def __init__(self, n_components=2, contamination=0.1):
        self.n_components = n_components
        self.contamination = contamination
    
    def fit(self, data):
        self.pca_model_ = PCA(n_components=self.n_components)
        self.pca_model_.fit(data)

        low_dim_data = self.pca_model_.transform(data)
        new_data = self.pca_model_.inverse_transform(low_dim_data)

        distances = np.linalg.norm(data - new_data, axis=1)

        self.threshold_ = -np.percentile(distances, 100 - self.contamination * 100)

        return self
        
    def decision_function(self, data):
        low_dim_data = self.pca_model_.transform(data)
        new_data = self.pca_model_.inverse_transform(low_dim_data)

        return - np.linalg.norm(data - new_data, axis=1)
    
    def predict(self, data):
        scores = self.decision_function(data)
        return np.where(scores > self.threshold_, +1, -1)

In [ ]:
model = PCAAnomalyDetector(n_components=1)
model.fit(data_blobs)

plot_level_lines(model, data_blobs)

Плюсы:
  * простой метод

Минусы:
  * использует линейную аппроксимацию данных

In [ ]:
model = PCAAnomalyDetector(n_components=12, contamination=0.01)

model.fit(train_X)
predictions_pca = -model.decision_function(test_X)

labels_pca = model.predict(test_X)

In [ ]:
plot_results(test_y, predictions_pca)

metrics_pca = show_metrics(test_y, (labels_pca < 0) * 1)

<br>

## Бинарная классификация

Воспользуемся логистической регрессией

In [ ]:
from sklearn.linear_model import LogisticRegression


estimator = LogisticRegression(class_weight=None)

grid = {
    "C" : np.logspace(-3, +3, num=10)
}

Будем стараться честно построить модель

In [ ]:
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import make_scorer

f1_scorer = make_scorer(f1_score)

Будем использовать модификацию $k$-fold валидации, которая **специально сохраняет баланс**
классов -- так называемый стратифицированный $k$-fold.

In [ ]:
from sklearn.model_selection import StratifiedKFold

st_kfold = StratifiedKFold(n_splits=5)

Оценим лог-регрессию с кросс валидацией

In [ ]:
cv_grid = GridSearchCV(estimator, grid, scoring=f1_scorer, cv=st_kfold, n_jobs=1)

cv_grid.fit(train_X, train_y) ;

Оценим метрики для самой лучшей модели по валидации

In [ ]:
logistic_naive = cv_grid.best_estimator_

logistic_naive_test_pred = logistic_naive.predict(test_X)

metrics_logistic_naive = show_metrics(test_y, logistic_naive_test_pred)

In [ ]:
metrics_logistic_naive

Можно ли улучшить `precision` и `recall`?

<br>

### Балансировка и Ресэмплинг

#### Балансировка лосса

В базовом ("наивном") случае лог-регрессия использует лог лосс:
* на выборке $(x_i, y_i)_{i=1}^m$ с $y_i \in \{\pm 1\}$ решается задача
\begin{aligned}
  & \underset{\beta_0, \beta}{\text{минимизировать}}
    & & \tfrac12 \|\beta\|_2^2
        + C \sum_{i=1}^m l\bigl(y_i, f(x_i)\bigr)
        \,, \\
  & & & l(y, p) = \log \bigl(1 + \exp\{- y p \}\bigr)
        \,, \\
  & & & f(x) = x^{\rm T} \beta + \beta_0
\end{aligned}

Но можно минимизировать функцию потерь, взвешенный по меткам класса:
\begin{aligned}
  & \underset{\beta_0, \beta}{\text{минимизировать}}
    & & \tfrac12 \|\beta\|_2^2
        + C w_+ \sum_{i\colon y_i = +1} l\bigl(+1, f(x_i)\bigr)
        + C w_- \sum_{i\colon y_i = -1} l\bigl(-1, f(x_i)\bigr)
        \,, \\
  & & & l(y, p) = \log \bigl(1 + \exp\{- y p \}\bigr)
        \,, \\
  & & & f(x) = x^{\rm T} \beta + \beta_0
\end{aligned}

* в случае "наивного" взвешивания $w_+ = w_- = 1$.

* в случае "сбалансированного" взвешивания $w_+ = \tfrac{m}{2 n_+}$ и $w_- = \tfrac{m}{2 n_-}$.

In [ ]:
estimator = LogisticRegression(class_weight = "balanced")

cv_grid = GridSearchCV(estimator, grid, scoring=f1_scorer, cv = st_kfold, n_jobs = 1)

cv_grid.fit(train_X, train_y) ;

In [ ]:
logistic_balanced = cv_grid.best_estimator_

logistic_balanced_test_pred = logistic_balanced.predict(test_X)

metrics_logistic_balanced = show_metrics(test_y, logistic_balanced_test_pred)

In [ ]:
metrics_logistic_balanced

#### Ресэмплинг

Основная идея -- сбалансировать классы в обучающей выборке добавлением наблюдений или изменением их веса.

Рассмотрим технику прореживания доминирующего класса (*undersampling*)

Для достижения баланса первая техника физически отбрасывает долю наблюдений доминирующего класса `0`.

In [ ]:
from sklearn.utils import check_random_state, safe_indexing


def undersample(X, y, ratio=20, pos_label=1, random_state=None):
    random_state = check_random_state(random_state)

    # отбрасываем случайную долю наблюдений доминирующего класса
    class_major_index = np.flatnonzero(y != pos_label)

    n_major = int(len(class_major_index) / ratio)
    class_major_index = random_state.permutation(class_major_index)
    class_major_index = class_major_index[:n_major]

    # выбираем все примеры минорного класса
    class_minor_index = np.flatnonzero(y == pos_label)

    # составляем новую (временную) обучающую выборку
    indices = np.r_[class_major_index, class_minor_index]

    return safe_indexing(X, indices), safe_indexing(y, indices)

Прореживание доминирующего класса
* может сильно уменьшить количество наблюдений в итоговой обучающей
выборке, что негативно сказывается на обученном классификаторе:
    * может плохо обобщить из-за нехватки данных;
    * сам алгоритм обучения моджет очень чувствителен к изменению выборки.


* снижает затраты на обучение модели, позволяя тренировать больше классификаторов

In [ ]:
estimator = LogisticRegression(class_weight=None)

Выделим из текущего трейна валидацоннный набор

In [ ]:
dev_X, val_X, dev_y, val_y = train_test_split(train_X, train_y, test_size=0.25,
                                              stratify=train_y, random_state=321)

Проведём валидацию модели с андерсэмплингом вручную

In [ ]:
from sklearn.model_selection import ParameterGrid
from sklearn.base import clone

n_resamples, ratio = 7, 10
par_grid = ParameterGrid(grid)

results_grid = []
for par in par_grid:

    results_resample = []
    for b in range(n_resamples):
        und_X, und_y = undersample(dev_X, dev_y, ratio, pos_label=1, random_state=None)

        cv_estimator = clone(estimator).set_params(**par)
        cv_estimator.fit(und_X, und_y)
        
        cv_val_pred = cv_estimator.predict(val_X)
        results_resample.append(show_metrics(val_y, cv_val_pred))

    results_resample = pd.concat(results_resample, axis=1).T

    results_grid.append((par, results_resample.mean().rename("mean")))

Выбираем $F_1$ метрики и находим наилучшую модель

In [ ]:
f1_scores = [(par, met["f1_score"]) for par, met in results_grid]

best_par_, _ = f1_scores[np.argmax([f1 for par, f1 in f1_scores])]

Честно делаем андерсэмплинг и обучаем модель заново

In [ ]:
und_X, und_y = undersample(train_X, train_y, ratio, pos_label=1, random_state=None)

logistic_undersample = clone(estimator).set_params(**best_par_)
logistic_undersample.fit(und_X, und_y)

Считаем метрики

In [ ]:
logistic_undersample_test_pred = logistic_undersample.predict(test_X)

metrics_logistic_undersample = show_metrics(test_y, logistic_undersample_test_pred)

In [ ]:
metrics_logistic_undersample

Сравним метрики

In [ ]:
all_metrics = pd.concat(dict([
    ("elliptic", metrics_elliptic),
    ("isolation", metrics_isolation),
    ("svm", metrics_svm),
    ("pca", metrics_pca),
    ("logistic_naive", metrics_logistic_naive),
    ("logistic_balanced", metrics_logistic_balanced),
    ("logistic_undersample", metrics_logistic_undersample)
]), axis=1)

all_metrics

<br>